In [1]:
using Turing
using LimberJack
using CSV
using NPZ
using Plots
using FITSIO
using Plots
using LinearAlgebra
using Distributed
using StatsPlots
using BenchmarkTools

In [2]:
Threads.nthreads()

4

In [3]:
files = npzread("../data/DESY1_cls/Cls_meta.npz")
Cls_meta = cls_meta(files);

In [4]:
cov_tot = files["cov"]
inv_cov_tot = files["inv_cov"]
data_vector = files["cls"];

In [5]:
nuisances = Dict("b0" => 1.41,
                 "b1" => 1.62,
                 "b2" => 1.60,
                 "b3" => 1.92,
                 "b4" => 2.00)

cosmology = LimberJack.Cosmology(0.25, 0.05, 0.67, 0.96, 0.81,
                                 tk_mode="EisHu",
                                 Pk_mode="Halofit");

In [6]:
@benchmark Theory(cosmology, nuisances, Cls_meta, files)

BenchmarkTools.Trial: 
  memory estimate:  706.20 MiB
  allocs estimate:  43274366
  --------------
  minimum time:     1.415 s (6.03% GC)
  median time:      1.444 s (5.92% GC)
  mean time:        1.437 s (6.08% GC)
  maximum time:     1.447 s (6.59% GC)
  --------------
  samples:          4
  evals/sample:     1

In [7]:
@benchmark Theory_parallel(cosmology, nuisances, Cls_meta, files)

BenchmarkTools.Trial: 
  memory estimate:  706.20 MiB
  allocs estimate:  43274408
  --------------
  minimum time:     1.240 s (12.69% GC)
  median time:      1.279 s (13.62% GC)
  mean time:        1.275 s (13.72% GC)
  maximum time:     1.303 s (14.91% GC)
  --------------
  samples:          4
  evals/sample:     1

In [20]:
@model function model(data_vector::Vector{Float64})
    Ωm ~ Uniform(0.1, 0.5)
    h = 0.67 #~ Uniform(0.6, 0.8)
    s8 ~ Uniform(0.6, 1.0)
    b0 = 1.41 #~ Uniform(0.8, 3.0)
    b1 = 1.62 #~ Uniform(0.8, 3.0)
    b2 = 1.60 #~ Uniform(0.8, 3.0)
    b3 = 1.92 #~ Uniform(0.8, 3.0)
    b4 = 2.00 #~ Uniform(0.8, 3.0)
    
    nuisances = Dict("b0" => b0,
                     "b1" => b1,
                     "b2" => b2,
                     "b3" => b3,
                     "b4" => b4)
    
    cosmology = LimberJack.Cosmology(Ωm, 0.05, h, 0.96, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    
    theory = vcat(Theory(cosmology, nuisances, Cls_meta, files)...)
    data_vector ~ MvNormal(theory, cov_tot)
    return data_vector
end;

Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64


In [19]:
@model function model_parallel(data_vector::Vector{Float64})
    Ωm ~ Uniform(0.1, 0.5)
    h = 0.67 #~ Uniform(0.6, 0.8)
    s8 ~ Uniform(0.6, 1.0)
    b0 = 1.41 #~ Uniform(0.8, 3.0)
    b1 = 1.62 #~ Uniform(0.8, 3.0)
    b2 = 1.60 #~ Uniform(0.8, 3.0)
    b3 = 1.92 #~ Uniform(0.8, 3.0)
    b4 = 2.00 #~ Uniform(0.8, 3.0)
    
    nuisances = Dict("b0" => b0,
                     "b1" => b1,
                     "b2" => b2,
                     "b3" => b3,
                     "b4" => b4)
    
    cosmology = LimberJack.Cosmology(Ωm, 0.05, h, 0.96, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    
    theory = vcat(Theory_parallel(cosmology, nuisances, Cls_meta, files)...)
    data_vector ~ MvNormal(theory, cov_tot)
    return data_vector
end;

Float64


In [10]:
iterations = 100
nadapts = 20
nchains = 4

4

In [14]:
chains_treads = sample(model(data_vector), MH(), MCMCThreads(), iterations,
                     nchains, nadapt=nadapts, progress=true, save_state=true)

Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64


LoadError: TaskFailedException

[91m    nested task error: [39mTaskFailedException
    Stacktrace:
     [1] [0m[1mwait[22m
    [90m   @ [39m[90m./[39m[90m[4mtask.jl:334[24m[39m[90m [inlined][39m
     [2] [0m[1mthreading_run[22m[0m[1m([22m[90mfunc[39m::[0mFunction[0m[1m)[22m
    [90m   @ [39m[90mBase.Threads[39m [90m./[39m[90m[4mthreadingconstructs.jl:38[24m[39m
     [3] [0m[1mmacro expansion[22m
    [90m   @ [39m[90m./[39m[90m[4mthreadingconstructs.jl:97[24m[39m[90m [inlined][39m
     [4] [0m[1mmacro expansion[22m
    [90m   @ [39m[90m~/.julia/packages/AbstractMCMC/oou1a/src/[39m[90m[4msample.jl:304[24m[39m[90m [inlined][39m
     [5] [0m[1m(::AbstractMCMC.var"#33#44"{Bool, Base.Pairs{Symbol, Any, Tuple{Symbol, Symbol, Symbol}, NamedTuple{(:chain_type, :nadapt, :save_state), Tuple{UnionAll, Int64, Bool}}}, Int64, Int64, Vector{Any}, Vector{UInt64}, Vector{DynamicPPL.Sampler{MH{(), NamedTuple{(), Tuple{}}}}}, Vector{Dynamic

In [15]:
chains_threads_parallel = sample(model_parallel(data_vector), MH(), MCMCThreads(), iterations,
                            nchains, nadapt=nadapts, progress=true, save_state=true)

Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64
Float64


LoadError: TaskFailedException

[91m    nested task error: [39mTaskFailedException
    Stacktrace:
     [1] [0m[1mwait[22m
    [90m   @ [39m[90m./[39m[90m[4mtask.jl:334[24m[39m[90m [inlined][39m
     [2] [0m[1mthreading_run[22m[0m[1m([22m[90mfunc[39m::[0mFunction[0m[1m)[22m
    [90m   @ [39m[90mBase.Threads[39m [90m./[39m[90m[4mthreadingconstructs.jl:38[24m[39m
     [3] [0m[1mmacro expansion[22m
    [90m   @ [39m[90m./[39m[90m[4mthreadingconstructs.jl:97[24m[39m[90m [inlined][39m
     [4] [0m[1mmacro expansion[22m
    [90m   @ [39m[90m~/.julia/packages/AbstractMCMC/oou1a/src/[39m[90m[4msample.jl:304[24m[39m[90m [inlined][39m
     [5] [0m[1m(::AbstractMCMC.var"#33#44"{Bool, Base.Pairs{Symbol, Any, Tuple{Symbol, Symbol, Symbol}, NamedTuple{(:chain_type, :nadapt, :save_state), Tuple{UnionAll, Int64, Bool}}}, Int64, Int64, Vector{Any}, Vector{UInt64}, Vector{DynamicPPL.Sampler{MH{(), NamedTuple{(), Tuple{}}}}}, Vector{Dynamic

In [16]:
chains_parallel = sample(model_parallel(data_vector), MH(), iterations, nchains,
                         nadapt=nadapts, progress=true, save_state=true)

LoadError: MethodError: no method matching sample(::DynamicPPL.Model{var"#3#4", (:data_vector,), (), (), Tuple{Vector{Float64}}, Tuple{}}, ::MH{(), NamedTuple{(), Tuple{}}}, ::Int64, ::Int64; nadapt=20, progress=true, save_state=true)
[0mClosest candidates are:
[0m  sample(::AbstractMCMC.AbstractModel, ::Turing.Inference.InferenceAlgorithm, [91m::AbstractMCMC.AbstractMCMCParallel[39m, ::Integer, [91m::Integer[39m; kwargs...) at ~/.julia/packages/Turing/uAz5c/src/inference/Inference.jl:181
[0m  sample(::AbstractMCMC.AbstractModel, ::Turing.Inference.InferenceAlgorithm, ::Integer; kwargs...) at ~/.julia/packages/Turing/uAz5c/src/inference/Inference.jl:126
[0m  sample(::AbstractMCMC.AbstractModel, [91m::AbstractMCMC.AbstractSampler[39m, [91m::AbstractMCMC.AbstractMCMCParallel[39m, ::Integer, [91m::Integer[39m; kwargs...) at ~/.julia/packages/AbstractMCMC/oou1a/src/sample.jl:34
[0m  ...

In [ ]:
describe(NUTS_chains)[1]

In [ ]:
NUTS_chains.value

In [ ]:
plot(new_chain)